In [ ]:
# using Pkg
# Pkg.activate(".")
# Pkg.instantiate()
# Pkg.add("SQLite")
# Pkg.add("DBInterface")
# Pkg.add("JSON3")
# Pkg.add("EasyConfig")
# Pkg.add("MurmurHash3")
# Pkg.add("TextAnalysis")
# Pkg.add("UUIDs")
# Pkg.add("PooledArrays")
include("src/lisa_store.jl")

using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays

db = Graph.DB("lisa.db")
hll = SetCore.HllSet{10}()

In [ ]:
p = 10
hll = SetCore.HllSet{p}()
Store.book_file(db, "/home/alexmy/JULIA/DEMO/raw/eu/", hll)

In [ ]:
using UUIDs
# parent::String, type::String,processor_id::String, status::String, lock_uuid::String
uuid = string(uuid4())
df = Graph.set_lock!(db, "/home/alexmy/JULIA/DEMO/raw/eu", "csv", "book_file", "ingest_csv", "waiting", "waiting", uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row.id,row.parent, row.item, row.a_type, row.processor_id, row.status, row.lock_uuid)     
    col_uuid = string(uuid4())
    # db::Graph.DB, file::Graph.Assignment, lock_uuid::String; p::Int=10    
    Store.ingest_csv(db, assign, col_uuid; limit=10000, offset=10)
end

In [ ]:
using EasyConfig
Store.commit(db, "lisa_arch.hdf5", "Alex Mylnikov", "alexmy@lisa-park.com", "commit 2", Config())

In [ ]:
using HDF5
arch = h5open("lisa_arch.hdf5")

In [ ]:
close(arch)

In [ ]:
using HDF5

file = h5open("lisa_arch.hdf5", "r")
data_out = Dict()
Store.read_datasets(file, data_out, "/b6b833df-2fab-490a-af20-52b62b047626/nodes/")
println(data_out)
# Close the file
close(file)

In [ ]:
Graph.unset_lock!(db, :, :)